I am going to attempt to produce a melody generator for the turnaround of Joe Henderson's "Isotope." The changes to the last two bars are "C7, A7, Gb7, Eb7." To start, I'll just do three note shapes made from the major triads of each chord. I'll write a function to choose notes in near proximity so there aren't big jumps. There will be an element of randomness, however, pertaining mainly to decisions of whether to move up or down. 

Let's map each note of the guitar to one number. Mapping notes to their respective MIDI numbers, let's start with low E (E2, 82.41Hz) as '28' and go all the way to a high C on the first string (C6, 1047Hz) as '72'.

In [2]:
# our set of all possible notes
guitar_notes = range(28,73)

In [8]:
# now make sets of notes pertaining to each triad/chord
C = [28,31,36,40,43,48,52,55,60,64,67,72]
A = []
Gb = []
Eb = []

# since the other three sets of notes are separated by minor thirds, or multiples of three chromatic steps,
# I can produce the other three sets of notes by shifting my first set accordingly
for i in range(0,len(C)):
    A.append(C[i] - 3)
    Gb.append(C[i] - 6)
    Eb.append(C[i] + 3)
# now we just adjust the sets so they fit our chosen range of the guitar
A = A[1:]
Gb = Gb[2:]
Gb.append(42)
Eb = Eb[:-1]

print C,A,Gb,Eb

[28, 31, 36, 40, 43, 48, 52, 55, 60, 64, 67, 72] [28, 33, 37, 40, 45, 49, 52, 57, 61, 64, 69] [30, 34, 37, 42, 46, 49, 54, 58, 61, 66, 42] [31, 34, 39, 43, 46, 51, 55, 58, 63, 67, 70]


In [8]:
# note to self:
# difference between accessing an attribute, and calling a function
# hey.you vs hey.you()

In [9]:
# Now, let's try to write our function

In [99]:
import numpy as np

def melody_writer():

    melody = []
    start = np.random.uniform()
    # we'll randomly start at C4, E4, or G4, which have indices 5,6,7 of the C set respectively
    if start < float(1)/3:
        index = 5
    elif start < float(2)/3:
        index = 6
    else:
        index = 7
    
    melody.append(C[index])
    for i in range(0,2):
        flip = np.random.uniform()
        if flip > .5:
            index += 1
        else:
            index -= 1
        melody.append(C[index])
    #print melody[-1]
    # now we move on to the remaining three triads
    # find the nearest two notes
    remaining_sets = [A,Gb,Eb]
    for key in remaining_sets:
        triad = list(key)
        # I find the note closest to our previous note
        # remove the current note so we always move to a different note
        # when we change keys
        if melody[-1] in triad:
            triad.remove(melody[-1])
        closest = min(triad, key=lambda x:abs(x-melody[-1]))
        #print triad
        #print closest
        melody.append(closest)
        
        for i in range(0,2):
            index = key.index(melody[-1])
            flip = np.random.uniform()
            if index == 0:
                index += 1
            if index == len(key) - 1:
                index -= 1
            if flip > .5:
                index += 1
            else:
                index -= 1
            melody.append(key[index])
    
    print melody
    melody_notes = []
    # now we convert the numbers into notes with octave numbers for display
    notes = ['C','C#','D','Eb','E','F','F#','G','G#','A','Bb','B']
    for note in melody:
        octave = 0
        while note >= 12:
            note -= 12
            octave += 1
        melody_notes.append(str(notes[note]) + str(octave))
    print melody_notes
    return melody
melody = melody_writer()

[48, 52, 55, 57, 52, 49, 46, 49, 54, 55, 51, 55]
['C4', 'E4', 'G4', 'A4', 'E4', 'C#4', 'Bb3', 'C#4', 'F#4', 'G4', 'Eb4', 'G4']


In [117]:
melody = [48, 52, 55, 57, 52, 49, 46, 49, 54, 55, 51, 46]

In [121]:
# create your MIDI object
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track

time = 0    # start at the beginning
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, 120)

# add some notes
channel = 0
volume = 100

beats = [0,0.6,1,2,2.6,3,4,4.6,5,6,6.6,7]
index = 0

for note in melody:
    pitch = note          
    time = beats[index]      # start on beat 0
    duration = 1         # 1 beat long
    mf.addNote(track, channel, pitch, time, duration, volume)
    index += 1

with open("output.mid", 'wb') as outf:
    mf.writeFile(outf)

In [89]:
import numpy as np
from midiutil.MidiFile import MIDIFile

def melody_midi_writer():

    melody = []
    start = np.random.uniform()
    # we'll randomly start at C4, E4, or G4, which have indices 5,6,7 of the C set respectively
    if start < float(1)/3:
        index = 5
    elif start < float(2)/3:
        index = 6
    else:
        index = 7
    
    melody.append(C[index])
    for i in range(0,2):
        flip = np.random.uniform()
        if flip > .5:
            index += 1
        else:
            index -= 1
        melody.append(C[index])
    #print melody[-1]
    # now we move on to the remaining three triads
    # find the nearest two notes
    remaining_sets = [A,Gb,Eb]
    for key in remaining_sets:
        triad = list(key)
        # I find the note closest to our previous note
        # remove the current note so we always move to a different note
        # when we change keys
        if melody[-1] in triad:
            triad.remove(melody[-1])
        closest = min(triad, key=lambda x:abs(x-melody[-1]))
        #print triad
        #print closest
        melody.append(closest)
        
        for i in range(0,2):
            index = key.index(melody[-1])
            flip = np.random.uniform()
            if index == 0:
                index += 1
            if index == len(key) - 1:
                index -= 1
            if flip > .5:
                index += 1
            else:
                index -= 1
            melody.append(key[index])
            
            
    # create your MIDI object
    mf = MIDIFile(1)     # only 1 track
    track = 0   # the only track

    time = 0    # start at the beginning
    mf.addTrackName(track, time, "Sample Track")
    mf.addTempo(track, time, 120)

    # add some notes
    channel = 0
    volume = 100
    
    timer = 0
    
    for note in melody:
        pitch = note          # C5 (middle C)
        time = timer             # start on beat 0
        duration = 1         # 1 beat long
        mf.addNote(track, channel, pitch, time, duration, volume)
        timer += 1
    
    with open("output.mid", 'wb') as outf:
        mf.writeFile(outf)    
    
    #print melody
    melody_notes = []
    # now we convert the numbers into notes with octave numbers for display
    notes = ['C','C#','D','Eb','E','F','F#','G','G#','A','Bb','B']
    for note in melody:
        octave = 0
        while note >= 12:
            note -= 12
            octave += 1
        melody_notes.append(str(notes[note]) + str(octave))
    return melody_notes
melody_midi_writer()

['C4', 'G3', 'C4', 'C#4', 'A3', 'C#4', 'Bb3', 'C#4', 'F#4', 'G4', 'Eb4', 'G4']

NOTE TO SELF: WATCH STEVE COLEMAN'S ELEMENTS OF ONES

In [103]:
# NOTE TO SELF: change the function so it doesn't 
# ALWAYS grab the nearest note when switching chords

In [104]:
# Wow, I rike dis
sample_melody = [48, 52, 55, 57, 52, 49, 46, 49, 54, 55, 51, 55]